<span style="font-size: 220%;color:#1155cc">Pytorch_9_Proste sieci neuronowe_pytorch-custom-nn-modules
    
<span style="font-size: 180%;color:red"> 17.09.2020   

https://github.com/jcjohnson/pytorch-examples#pytorch-custom-nn-modules


In [1]:
import torch

Odpalam karte graficzną GPU

In [2]:
#device = torch.device('cpu') # obliczenia robie na CPU
device = torch.device('cuda') # obliczenia robie na GPU

### Dane
- N to wielkość partii (ilośc rekordów, wielkość próbki); 
- D_in jest wymiarem wejściowym (liczba zmiennych opisujących); 
- H jest ukrytym wymiarem (ukryta warstwa); 
- D_out jest wymiarem wyjściowym(ilość zmiennych wynikowych). 

## <span style="color:red">Przyjmuje 10 zmiennych wynikowych! Coś nowego!</span>

In [3]:
N, D_in, H, D_out = 64, 1000, 100, 10

Powyżej zostały określone parametry aby tensor zmiennych niezależnych i tensor wynikowy były odpowiednie 

x: 64 obserwacji i 1000 zmiennych

y: 64 obserwacji i 10 zmiennych

### Tworzy losowe dane wejściowe i wyjściowe 

In [4]:
x = torch.randn(N, D_in, device=device)
y = torch.randn(N, D_out, device=device)

In [5]:
x.shape

torch.Size([64, 1000])

In [6]:
print('x ',x.size())
print('y ',y.size())

x  torch.Size([64, 1000])
y  torch.Size([64, 10])


### Niestandardowe funkcje autograd

In [7]:
class MyReLU(torch.autograd.Function):
  """
   Możemy zaimplementować własne niestandardowe funkcje autograd poprzez podklasowanie
   torch.autograd. Funkcja i realizacja przejść do przodu i wstecz
   które działają na Tensorach.
  """
  
  def forward(ctx, x):
    """
     W przekazie do przodu otrzymujemy obiekt kontekstowy i Tensor zawierający
     Wejście; musimy zwrócić Tensor zawierający dane wyjściowe i możemy użyć
     obiekt kontekstowy do buforowania obiektów do użycia w przebiegu wstecznym.
    """
    ctx.save_for_backward(x)
    return x.clamp(min=0)

  
  def backward(ctx, grad_output):
    """
     W wstecznym przebiegu otrzymujemy obiekt kontekstu i Tensor zawierający
     gradient straty w stosunku do produkcji wytworzonej podczas
     podanie do przodu. Możemy pobrać buforowane dane z obiektu kontekstu i musimy
     obliczyć i zwrócić gradient straty w odniesieniu do danych wejściowych do
     funkcja przewijania do przodu.
    """
    x, = ctx.saved_tensors
    grad_x = grad_output.clone()
    grad_x[x < 0] = 0
    return grad_x

### Tworzy wagi losowe dla x i y <span style="color:red">(requires_grad=True)</span>

Utwórzenie losowych tensorów dla wag; ustawienie 'requires_grad=True' oznacza, że my
chce obliczyć gradienty dla tych Tensorów podczas przebiegu wstecznego.

In [8]:
w1 = torch.randn(D_in, H, device=device, requires_grad=True)
w2 = torch.randn(H, D_out, device=device, requires_grad=True)

In [9]:
w1

tensor([[ 0.4146,  1.2637, -0.0543,  ...,  0.4120, -0.6752, -0.0125],
        [ 1.0447, -0.7799,  0.1340,  ..., -1.1006,  0.8972, -0.7874],
        [ 1.1188,  1.0270,  1.3485,  ..., -1.5144, -1.3482, -0.3158],
        ...,
        [ 1.0269,  0.2173, -1.4205,  ...,  1.9121, -0.6447, -0.4521],
        [ 0.1893,  0.3817, -1.1482,  ...,  0.0196, -0.7121, -1.0367],
        [ 1.4844, -0.6829, -1.3740,  ..., -0.2077,  1.5886, -1.2003]],
       device='cuda:0', requires_grad=True)

In [10]:
print('w1 dla x: ', w1.shape)
print('w2 dla y: ', w2.shape)

w1 dla x:  torch.Size([1000, 100])
w2 dla y:  torch.Size([100, 10])


### Definiowanie nauki
     Przekaz do przodu: oblicz przewidywane y przy użyciu operacji na Tensorach; nazywamy nasz
     niestandardowa implementacja ReLU za pomocą funkcji MyReLU.apply


In [11]:
learning_rate = 1e-6
for t in range(500):
 

    y_pred = x.mm(w1).clamp(min=0).mm(w2)
  
      # Funkcja błędu r2 dla drukowana dla każdej epoki
    loss = (y_pred - y).pow(2).sum()
    print('Epoka:',t, loss.item())
    
       #KROK 1
       # Użyj programu autograd, aby obliczyć przebieg wsteczny. 
    loss.backward()

       #KROK 2
       # Zaktualizuj wagi przy użyciu spadku gradientu.
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad

       #KROK 3
       # Ręcznie zeruj gradienty po przejściu do tyłu
        w1.grad.zero_()
        w2.grad.zero_()
    
    

Epoka: 0 41059232.0
Epoka: 1 37348612.0
Epoka: 2 31590172.0
Epoka: 3 22418816.0
Epoka: 4 13508976.0
Epoka: 5 7458933.0
Epoka: 6 4255358.5
Epoka: 7 2692282.75
Epoka: 8 1906821.125
Epoka: 9 1466367.0
Epoka: 10 1185245.0
Epoka: 11 985638.75
Epoka: 12 833117.875
Epoka: 13 711908.875
Epoka: 14 613022.5
Epoka: 15 531202.9375
Epoka: 16 462860.3125
Epoka: 17 405257.9375
Epoka: 18 356328.25
Epoka: 19 314612.5625
Epoka: 20 278731.78125
Epoka: 21 247742.5
Epoka: 22 220889.90625
Epoka: 23 197486.671875
Epoka: 24 177026.53125
Epoka: 25 159073.46875
Epoka: 26 143270.75
Epoka: 27 129307.75
Epoka: 28 116933.7890625
Epoka: 29 105944.296875
Epoka: 30 96165.828125
Epoka: 31 87433.2421875
Epoka: 32 79621.4921875
Epoka: 33 72613.546875
Epoka: 34 66317.1953125
Epoka: 35 60646.3515625
Epoka: 36 55527.12890625
Epoka: 37 50900.78125
Epoka: 38 46713.4765625
Epoka: 39 42919.375
Epoka: 40 39478.5078125
Epoka: 41 36362.92578125
Epoka: 42 33526.08203125
Epoka: 43 30939.609375
Epoka: 44 28579.26171875
Epoka: 45 2641

Epoka: 312 0.04385976493358612
Epoka: 313 0.0420772023499012
Epoka: 314 0.040371038019657135
Epoka: 315 0.0387333445250988
Epoka: 316 0.037167735397815704
Epoka: 317 0.03565699979662895
Epoka: 318 0.03422113135457039
Epoka: 319 0.032839443534612656
Epoka: 320 0.03151104971766472
Epoka: 321 0.030248286202549934
Epoka: 322 0.029025670140981674
Epoka: 323 0.02785465493798256
Epoka: 324 0.02673441544175148
Epoka: 325 0.025653187185525894
Epoka: 326 0.02461865544319153
Epoka: 327 0.023640302941203117
Epoka: 328 0.022685889154672623
Epoka: 329 0.021775636821985245
Epoka: 330 0.02089342102408409
Epoka: 331 0.020067088305950165
Epoka: 332 0.01926233619451523
Epoka: 333 0.01850433647632599
Epoka: 334 0.017769981175661087
Epoka: 335 0.01706218533217907
Epoka: 336 0.01638450287282467
Epoka: 337 0.01572566106915474
Epoka: 338 0.015109414234757423
Epoka: 339 0.014507804065942764
Epoka: 340 0.013934489339590073
Epoka: 341 0.013379087671637535
Epoka: 342 0.012855718843638897
Epoka: 343 0.012349041178